# 回归问题分析

In [1]:
import pandas as pd
import numpy as np
import numpy as np

In [2]:
x_5mer = pd.read_csv('/home/xlxiang/biostatistics/data/Rregression/x-2k_sequence_5mer.csv',header=None)
y = pd.read_csv('/home/xlxiang/biostatistics/data/Rregression/y.csv',index_col=0)
y = np.log2(y['TPM'].values+1)
x_5mer[1024]=y

In [3]:
from sklearn.model_selection import train_test_split
#划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(x_5mer.iloc[:,:1024], x_5mer.iloc[:,1024], random_state=2023)
print(len(X_train))
print(len(X_test))

41800
13934


In [4]:
#计算训练集和测试集的pearsonr和r2
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
def calculate_result(model,x_train, y_train,x_test, y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    train_pearsonr = pearsonr(y_train, y_pred_train)[0]
    test_pearsonr = pearsonr(y_test, y_pred_test)[0]
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    print("train_pearsonr: {0:.3f}, test_pearsonr: {1:.3f}".format(train_pearsonr,test_pearsonr))
    print("train_r2: {0:.3f}, test_r2: {1:.3f}".format(train_r2, test_r2))

    return None

## KNN

In [5]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
#KNN 寻参
params = {'n_neighbors': [10,15,20,25,30]}
model_knn = GridSearchCV(KNeighborsRegressor(),param_grid=params,cv=10,n_jobs=3)
model_knn.fit(X_train, y_train)
print("Best parameters: {}".format(model_knn.best_params_))
print("Best cross-validation score: {:.2f}".format(model_knn.best_score_))
calculate_result(model_knn,X_train,y_train,X_test,y_test)

Best parameters: {'n_neighbors': 20}
Best cross-validation score: 0.45
train_pearsonr: 0.714, test_pearsonr: 0.680
train_r2: 0.506, test_r2: 0.462


## linear regression

In [6]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_train, y_train)
calculate_result(lr,X_train,y_train,X_test,y_test)

train_pearsonr: 0.677, test_pearsonr: 0.657
train_r2: 0.459, test_r2: 0.432


## Lasso

In [9]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
params = {'alpha': [1e-7,1e-6,0.00001,0.0001]}
model_lasso = GridSearchCV(Lasso(max_iter=2000),param_grid=params,cv=10,n_jobs=3)
model_lasso.fit(X_train, y_train)
print("Best parameters: {}".format(model_lasso.best_params_))
print("Best cross-validation score: {:.2f}".format(model_lasso.best_score_))
calculate_result(model_lasso,X_train,y_train,X_test,y_test)

Best parameters: {'alpha': 1e-06}
Best cross-validation score: 0.43
train_pearsonr: 0.674, test_pearsonr: 0.659
train_r2: 0.455, test_r2: 0.435


/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.858e+02, tolerance: 3.679e+01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.583e+04, tolerance: 3.306e+01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/home/xlxiang/anaconda3/envs/tf_2.4/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of it

## ridge

In [9]:
from sklearn.linear_model import Ridge
params = {'alpha': [0.00001,0.0001, 0.001, 0.1, 1]}
model_ridge = GridSearchCV(Ridge(),param_grid=params,cv=10,n_jobs=3)
model_ridge.fit(X_train, y_train)
print("Best parameters: {}".format(model_ridge.best_params_))
print("Best cross-validation score: {:.2f}".format(model_ridge.best_score_))
calculate_result(model_ridge,X_train,y_train,X_test,y_test)

Best parameters: {'alpha': 0.001}
Best cross-validation score: 0.43
train_pearsonr: 0.674, test_pearsonr: 0.660
train_r2: 0.455, test_r2: 0.435


## SVR

In [10]:
from sklearn.svm import SVR
params = [
        {'kernel': ['linear'], 'C': [100, 1000, 5000]},
        {'kernel': ['rbf'], 'C': [50,100,200], 'gamma': [10, 1, 0.1]},
    ]
model_svr = GridSearchCV(SVR(),param_grid=params,cv=10,n_jobs=3)
model_svr.fit(X_train, y_train)
print("Best parameters: {}".format(model_svr.best_params_))
print("Best cross-validation score: {:.2f}".format(model_svr.best_score_))
calculate_result(model_svr,X_train,y_train,X_test,y_test)

Best parameters: {'C': 200, 'gamma': 10, 'kernel': 'rbf'}
Best cross-validation score: 0.45
train_pearsonr: 0.697, test_pearsonr: 0.677
train_r2: 0.481, test_r2: 0.452


## RF

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
# params = { 'n_estimators':randint(low=300,high=600),
#     'max_depth':randint(low=20,high=50),
#     'max_depth':randint(low=5,high=30)}
# model_rf = RandomizedSearchCV(RandomForestRegressor(),param_distributions=params,n_iter=50,cv=10,n_jobs=5)
model_rf = RandomForestRegressor(n_estimators=200,max_features=315,max_depth=29,min_samples_split=11,n_jobs=-1)
model_rf.fit(X_train, y_train)
# print("Best parameters: {}".format(model_rf.best_params_))
# print("Best cross-validation score: {:.2f}".format(model_rf.best_score_))
calculate_result(model_rf,X_train,y_train,X_test,y_test)

AttributeError: 'RandomForestRegressor' object has no attribute 'best_params_'

In [6]:
calculate_result(model_rf,X_train,y_train,X_test,y_test)

train_pearsonr: 0.962, test_pearsonr: 0.653
train_r2: 0.860, test_r2: 0.406
